In [229]:
test_annotations = {"a": [None, None, None, None, None, 1, 3, 0, 1, 0, 0, 2, 2, None, 2],
                    "b": [0, None, 1, 0, 2, 2, 3, 2, None, None, None, None, None, None, None],
                    "c": [None, None, 1, 0, 2, 3, 3, None, 1, 0, 0, 2, 2, None, 3]}
labels = [0, 1, 2, 3]

In [230]:
import pandas as pd
import math
import collections

In [237]:
def df2table(df, labels):
    n = len(labels)
    
    df_rows = []
    for idx, row in df.iterrows():
        labels = [0] * n
        for label in row:
            if not math.isnan(label):
                labels[int(label)] += 1
        df_rows.append(labels)
    
    return pd.DataFrame(df_rows, columns=labels)

In [238]:
df = pd.DataFrame(test_annotations)

In [239]:
df

,a,b,c
0,NaN,0.0,NaN
1,NaN,NaN,NaN
2,NaN,1.0,1.0
3,NaN,0.0,0.0
4,NaN,2.0,2.0
5,1.0,2.0,3.0
6,3.0,3.0,3.0
7,0.0,2.0,NaN
8,1.0,NaN,1.0
9,0.0,NaN,0.0


In [240]:
labels_per_instance = []

In [241]:
for i, row in df.iterrows():
    # Append the number of non-NaNs for each data instance
    num_nans = sum(math.isnan(k) for k in row)
    length = len(row)
    labels_per_instance.append(length - num_nans)

In [243]:
num_instances = df.shape[0]
fleiss_df = df2table(df, labels)

In [244]:
fleiss_df

,0,0,1,1
0,1,0,0,0
1,0,0,0,0
2,0,2,0,0
3,2,0,0,0
4,0,0,2,0
5,0,1,1,1
6,0,0,0,3
7,1,0,1,0
8,0,2,0,0
9,2,0,0,0


In [248]:
def proportion_label_per_category(df):
    num_assignments = list(df.sum(axis=0))
    normaliser = 1. / sum(num_assignments)
    return [normaliser * i for i in num_assignments]

def rater_agreement_extent(df):
    # PROBLEM HERE: Each element of returned list should be <= 1
    rater_agreement_ext = []
    
    total_labels = list(df.sum(axis=1))
        
    df2 = df ** 2
    total_labels_squared = list(df2.sum(axis=1))
        
    v1 = np.array(total_labels_squared)
    v2 = np.array(total_labels)
    summations = list(v1 - v2)
    
    final = []
    for i in range(len(total_labels)):
        try:
            normalise = 1. / (total_labels[i] * (total_labels[i]-1))
            final.append(normalise * summations[i])
        except ZeroDivisionError:
            normalise = 0.
        
    return final

In [249]:
prop_labels_per_cat = proportion_label_per_category(fleiss_df)
print(prop_labels_per_cat)

[0.2962962962962963, 0.18518518518518517, 0.3333333333333333, 0.18518518518518517]


In [250]:
rater_agreement_ext = rater_agreement_extent(fleiss_df)
print(rater_agreement_ext)

[1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0]


In [251]:
mean_P = (1. / num_instances) * sum(rater_agreement_ext)

In [252]:
mean_p = sum([i ** 2 for i in prop_labels_per_cat])

In [253]:
if mean_p == 1:
    mean_p = 1.

In [254]:
final = (mean_P - mean_p) / (1 - mean_p)

In [255]:
final

0.453932584269663

In [256]:
print(prop_labels_per_cat)
print()
print(rater_agreement_ext)

[0.2962962962962963, 0.18518518518518517, 0.3333333333333333, 0.18518518518518517]

[1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0]


In [156]:
df

,a,b,c
0,NaN,0.0,NaN
1,NaN,NaN,NaN
2,NaN,1.0,1.0
3,NaN,0.0,0.0
4,NaN,2.0,2.0
5,1.0,2.0,3.0
6,3.0,3.0,3.0
7,0.0,2.0,NaN
8,1.0,NaN,1.0
9,0.0,NaN,0.0


In [162]:
list(df.sum(axis=1))

[0.0, 0.0, 2.0, 0.0, 4.0, 6.0, 9.0, 2.0, 2.0, 0.0, 0.0, 4.0, 4.0, 0.0, 5.0]

In [159]:
df

,a,b,c
0,NaN,0.0,NaN
1,NaN,NaN,NaN
2,NaN,1.0,1.0
3,NaN,0.0,0.0
4,NaN,2.0,2.0
5,1.0,2.0,3.0
6,3.0,3.0,3.0
7,0.0,2.0,NaN
8,1.0,NaN,1.0
9,0.0,NaN,0.0
